<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/ML_cross_validation%2CGridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']]
target= wine['class'].to_numpy()

In [5]:
print(data.head())
print()
print(target)

   alcohol  sugar    pH
0      9.4    1.9  3.51
1      9.8    2.6  3.20
2      9.8    2.3  3.26
3      9.8    1.9  3.16
4      9.4    1.9  3.51

[0. 0. 0. ... 1. 1. 1.]


# validation set (검증테스트)
- 훈련세트에서 모델을 훈련하고 검증세트로 모델을 평가한다.
- 그리고 마지막에 테스트 세트에서 최종점수를 평가한다.

In [6]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [9]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class'].to_numpy()

In [18]:
print(data.shape)    # 특성 3개
print(target.shape)  # 1차원의 리스트

(6497, 3)
(6497,)


In [14]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)
# train_input 과 test_input 에 data의 특성을 7:3으로, target을 각각의 타겟에 7:3으로 넣고
# 랜덤하게 섞기

In [20]:
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [15]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)
# 섞여있는 train_input의 특성을 sub_input과 val_input에 적절하게 배분
# 즉, 훈련세트를 훈련세트와 검증세트로 분류

In [17]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [21]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input, val_target))

# 검증세트의 모델을 만들고 평가해보기
# val_input, val_target 을 사용해서 모델을 평가하는 것이다
# 이 모델은 훈련세트에 과대적합되어 있음
# 매개변수를 바꿔서 더 좋은 모델 찾자

0.9971133028626413
0.864423076923077


### cross validation (교차검증)
- 검증 세트를 만드느라 훈련세트가 줄었음

- 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안하다
- 교차검증을 사용하면 안정적인 검증점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있고 보통 더 많은 훈련데이터를 훈련에 사용할 수록 좋은 모델이 됨

In [22]:
from sklearn.model_selection import cross_validate
score = cross_validate(dt, train_input, train_target)
print(score)
 # 평가할 모델 객체를 첫번째 매개변수로, 그 다음 직접 검증세트를 떼어내지 않고 훈련세트 전체를 croos_validate() 함수에 전달\
 # cross_vaildate() 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환함
 # cross_vaildate() 함수는 기본적으로 5-폴드 교차검증을 수행
 # fit_time 은 모델을 훈련하는 시간 
 # socre_time 은 모델을 검증하는 시간
 # test_score 는 5개의 검증모델이 검증한 점수
 # 교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음

{'fit_time': array([0.01526833, 0.0135026 , 0.01337171, 0.01294732, 0.01313329]), 'score_time': array([0.00366306, 0.00361252, 0.00353217, 0.00625277, 0.00353956]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [24]:
import numpy as np
print(np.mean(score['test_score'])) # 검증폴드의 점수 / test_score 키에 담긴 5개의 점수를 평균
# 교차검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 검수를 가늠해 볼 수 있음

0.855300214703487


In [25]:
from sklearn.model_selection import StratifiedKFold
score = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(score['test_score']))
# 회귀모델일 경우 KFold 분할기를 사용
# 분류모델일 경우 타깃클라스를 골고루 나누기 위해 StratifiedKFold 사용

0.855300214703487


In [26]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # n_splits 매개변수는 몇(k) 폴드 교차 검증을 할지 정함
score = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(score['test_score']))

# 위는 훈련세트를 섞은 후 10- 폴드 교차검증을 수행하려면 다음과 같이 작성

0.8574181117533719


### Grid Search (그리드 서치)
- GridSearchCV 클래스는 친절하게 하이퍼파라미터 탐색과 교차검증을 한 번에 수행
- 별도로 cross_validate() 함수를 호출할 필요 없음

In [27]:
# 이어서 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델 찾아보자
# GridSearchCV 클래스를 임포트 하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 바꾸자
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}  # min_impurity_decrease ??

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs= -1)
# 0.0001 부터 5 까지 0.0001 씩 증가하는 5개의 값을 시도하겠다
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦

In [29]:
gs.fit(train_input, train_target)

# gs 객체에 fit() 메서드를 호출
# 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행함
# GridSearchCV의 cv 매개변수 기본값은 5 이다
# 따라서 min_impurity_decrease 값마다 5-폴드 교차검증을 수행함 => 5x5 =25 개의 모델을 훈련함
# n_jobs 기본 값은 1, -1로 지정하면 시스템에 있는 모든 코어를 사용함

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [30]:
# 교차검증에서 최적의 파라미터를 찾으면 전체 훈련세트로 모델을 다시 만들어야함
# 편리하게도 사이킷런의 그리드서치는 훈련이 끝나면 25개의 모델중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 
# 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다

dt = gs.best_estimator_      # 이 모델은 gs객체의 best_estimator_ 속성에 저장되있음 
print(dt.score(train_input, train_target))

0.9615162593804117


In [32]:
print(gs.best_params_) # 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되있음
# 여기서는 0.0001 이 가장 좋은 값으로 선택됨

{'min_impurity_decrease': 0.0001}


In [34]:
print(gs.cv_results_['mean_test_score'])
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장되어있음


[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [35]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])
# 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출가능
# 이 값이 최상의 검증점수를 만든 매개변수 조합이다

{'min_impurity_decrease': 0.0001}


- 1. 먼저 탐색할 매개변수를 지정
- 2. 그 다음 훈련세트에서 그리드서치를 수행한다
- 3. 그리하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다
- 4. 이 조합은 그리드 서치 객체에 저장 됨
- 5. 그리드 섳는 최상이 매개변수에서 (교차검증에 사용한 훈련세트가 아니라) 전체 훈련세트를 사용해 최종모델을 훈련함
- 6. 이 모델도 그리드 서치 객체에 저장 됨



In [36]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), # 총9개
          'max_depth' : range(5, 20, 1), # 15개
          'min_samples_split' : range(2, 100, 10)} # 10개의 값
# 이 매개변수로 수행할 교차검증횟수는 9 x 15 x 10 = 1,350 개
# 기본 5-폴트교차 검증을 수행하므로 모델의 수는 6,750 개

In [40]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
# 그리드 서치 실행

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [41]:
# 최상의 매개변수 조합을 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [42]:
# 최상의 교차검증점수 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [43]:
print(np.max(gs.cv_results_['mean_test_score']))
# GridSearchCV 클래스를 사용하므로 매개변수를 일일이 바꿔가며 교차검증을 수행하지 않아도 됨
# 원하는 매개변수 값을 나열하면 자동으로 교차검증을 수행해서 최상의 매개변수를 찾을 수 있음

0.8683865773302731


### Random Search (랜덤서치)
- 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데 이렇게 간격을 둔것에 특별한 근거는 없음
- 매개 변수의 값이 수치 일 때 범위나 간격을 미리 정하기 어려울 수 있음
- 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래 걸릴 수 있음
- 이럴 때 랜덤서치 사용하면 좋음

In [46]:
from scipy.stats import uniform, randint
# uniform 과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음
# 이를 '균등분포에서 샘플링 한다'고 함
# randint 는 정수 값을 uniform은 실수 값을 뽑음

In [47]:
rgen = randint(0,10)
rgen.rvs(10)
# randint 객체를 만들고 0 에서 10개의 숫자를 샘플링 

array([8, 7, 5, 1, 7, 1, 1, 7, 0, 8])

In [56]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  93, 108, 109,  95,  99,  96, 115, 105,  75]))

In [57]:
ugen = uniform(0,1)
ugen.rvs(10) 
# 0~1 사이 10개의 실수 추출

# randint 과 uniform 클래스 객체를 넘겨주고 총 몇번을 샘플링해서 최적의 매개변수를 찾으라고 명령 하는 것
# 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게하는게 좋음

array([0.10634576, 0.08076016, 0.66787572, 0.23014632, 0.1797004 ,
       0.44803092, 0.37609528, 0.92867087, 0.0027243 , 0.72721113])

In [58]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [60]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff13ae1efd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff13a4c2690>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff13ae1e690>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff13ae1ed90>},
                   random_state=42)

In [63]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [64]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [65]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
